In [41]:
import pandas as pd
import numpy as np
from src import configuration as config
import tensorflow as tf
import tensorflow_ranking as tfr
import tensorflow_recommenders as tfrs
import tensorflow_datasets as tfds

In [42]:
def stack_dict(inputs, fun=tf.stack):
    values = []
    for key in sorted(inputs.keys()):
      values.append(tf.cast(inputs[key], tf.float32))

    return fun(values, axis=-1)

In [43]:
# load the data
df = config.load_traindata_for_pointwise()
df = df.drop(columns=['cv_score'])
print(df.dtypes)
df.head()

dataset      int64
model       object
tuning      object
scoring     object
encoder     object
rank       float64
dtype: object


,dataset,model,tuning,scoring,encoder,rank
0,1169,KNC,model,ACC,BUCV2RGLMME,16.0
1,1169,KNC,model,ACC,BUCV2TE,14.0
2,1169,KNC,model,ACC,CBE,22.0
3,1169,KNC,model,ACC,CE,23.0
4,1169,KNC,model,ACC,CV10RGLMME,7.0


In [44]:
def create_encoder_rankings(df):
    # Group the DataFrame by 'dataset', 'model', 'tuning', and 'scoring' columns
    grouped_df = df.groupby(['dataset', 'model', 'tuning', 'scoring'])
    
    # Create a new DataFrame to store the results
    new_df = pd.DataFrame(columns=['dataset', 'model', 'tuning', 'scoring', 'encoder_rankings'])
    
    for group_keys, group_data in grouped_df:
        dataset, model, tuning, scoring = group_keys
        encoder_rankings = group_data.sort_values('rank', ascending=False)['encoder'].tolist()
        rankings = group_data.sort_values('rank', ascending=False)['rank'].tolist()
        new_row = {'dataset': dataset, 'model': model, 'tuning': tuning, 'scoring': scoring,
                   'encoder_rankings': [encoder_rankings], 'ranking': [rankings]}
        new_df = pd.concat([new_df, pd.DataFrame([new_row])], ignore_index=True)
    
    return new_df

# Your original DataFrame
data = {
    'dataset': [1169, 1169, 1169, 1169, 1169],
    'model': ['KNC', 'KNC', 'KNC', 'KNC', 'KNC'],
    'tuning': ['model', 'model', 'model', 'model', 'model'],
    'scoring': ['ACC', 'ACC', 'ACC', 'ACC', 'ACC'],
    'encoder': ['BUCV2RGLMME', 'BUCV2TE', 'CBE', 'CE', 'CV10RGLMME'],
    'rank': [16.0, 14.0, 22.0, 23.0, 7.0]
}

test_df = pd.DataFrame(data)

# Call the method to create the new DataFrame
new_dataframe = create_encoder_rankings(test_df)

# Display the new DataFrame
new_dataframe.head()

,dataset,model,tuning,scoring,encoder_rankings,ranking
0,1169,KNC,model,ACC,"[[CE, CBE, BUCV2RGLMME, BUCV2TE, CV10RGLMME]]","[[23.0, 22.0, 16.0, 14.0, 7.0]]"


In [45]:
df_listwise = create_encoder_rankings(df)
df_listwise.head()

,dataset,model,tuning,scoring,encoder_rankings,ranking
0,3,DTC,full,ACC,"[[DE, CBE, PBTE01, BE, OE, ME01E, ME10E, ME1E,...","[[4.0, 3.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,..."
1,3,DTC,full,AUC,"[[DE, CBE, PBTE01, BE, OE, ME01E, ME10E, ME1E,...","[[4.0, 3.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,..."
2,3,DTC,full,F1,"[[CBE, DE, PBTE01, BE, OE, ME01E, ME10E, ME1E,...","[[4.0, 3.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,..."
3,3,DTC,model,AUC,"[[DE, CBE, PBTE01, CV2TE, CV2RGLMME, CV5RGLMME...","[[25.0, 24.0, 23.0, 22.0, 21.0, 20.0, 19.0, 18..."
4,3,DTC,model,F1,"[[CBE, DE, PBTE01, CV2TE, CV2RGLMME, CV5RGLMME...","[[25.0, 24.0, 23.0, 22.0, 21.0, 20.0, 19.0, 18..."


In [46]:
df.head()

,dataset,model,tuning,scoring,encoder,rank
0,1169,KNC,model,ACC,BUCV2RGLMME,16.0
1,1169,KNC,model,ACC,BUCV2TE,14.0
2,1169,KNC,model,ACC,CBE,22.0
3,1169,KNC,model,ACC,CE,23.0
4,1169,KNC,model,ACC,CV10RGLMME,7.0


In [47]:
numerical_feature_names = ['dataset']
categorical_feature_names = ['model', 'tuning', 'scoring', 'encoder']
list_feature_names = ['encoder_rankings']
df = df.drop(columns=['rank'])
target = df_listwise.pop('ranking')

inputs = {}
for name, column in df.items():
  if type(column[0]) == str:
    dtype = tf.string
  elif (name in categorical_feature_names or
        name in numerical_feature_names):
    dtype = tf.int64
  else:
    dtype = tf.float32

  inputs[name] = tf.keras.Input(shape=(), name=name, dtype=dtype)
  
print(inputs)

{'dataset': <KerasTensor: shape=(None,) dtype=int64 (created by layer 'dataset')>, 'model': <KerasTensor: shape=(None,) dtype=string (created by layer 'model')>, 'tuning': <KerasTensor: shape=(None,) dtype=string (created by layer 'tuning')>, 'scoring': <KerasTensor: shape=(None,) dtype=string (created by layer 'scoring')>, 'encoder': <KerasTensor: shape=(None,) dtype=string (created by layer 'encoder')>}


In [48]:
preprocessed = []

# preprocessed.append(inputs['dataset'])
# preprocessed.append(inputs['rank'])

numeric_features = df[numerical_feature_names]

normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(stack_dict(dict(numeric_features)))

numeric_inputs = {}
for name in numerical_feature_names:
  numeric_inputs[name]=inputs[name]

numeric_inputs = stack_dict(numeric_inputs)
numeric_normalized = normalizer(numeric_inputs)

preprocessed.append(numeric_normalized)


for name in categorical_feature_names:
  vocab = sorted(set(df[name]))
  print(f'name: {name}')
  print(f'vocab: {vocab}\n')

  if type(vocab[0]) is str:
    lookup = tf.keras.layers.StringLookup(vocabulary=vocab, output_mode='one_hot')
  else:
    lookup = tf.keras.layers.IntegerLookup(vocabulary=vocab, output_mode='one_hot')

  x = inputs[name][:, tf.newaxis]
  x = lookup(x)
  preprocessed.append(x)

# # Convert list feature to separate categorical features
# for list_feature_name in list_feature_names:
#   unique_strings = set(np.concatenate(df[list_feature_name].values))
#   list_lookup = tf.keras.layers.StringLookup(vocabulary=unique_strings, output_mode='one_hot')

#   list_features = df[list_feature_name].apply(lambda x: tf.constant(x))
#   list_features = list_lookup(list_features)

#   # Append the list categorical features to the preprocessed list
#   preprocessed.append(list_features)
  
print(preprocessed)

name: model
vocab: ['DTC', 'KNC', 'LGBMC', 'LR', 'SVC']

name: tuning
vocab: ['full', 'model', 'no']

name: scoring
vocab: ['ACC', 'AUC', 'F1']

name: encoder
vocab: ['BE', 'BUCV10RGLMME', 'BUCV10TE', 'BUCV2RGLMME', 'BUCV2TE', 'BUCV5RGLMME', 'BUCV5TE', 'CBE', 'CE', 'CV10RGLMME', 'CV10TE', 'CV2RGLMME', 'CV2TE', 'CV5RGLMME', 'CV5TE', 'DE', 'DTEM10', 'DTEM2', 'DTEM5', 'ME01E', 'ME10E', 'ME1E', 'MHE', 'OE', 'OHE', 'PBTE0001', 'PBTE001', 'PBTE01', 'RGLMME', 'SE', 'TE', 'WOEE']

[<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'normalization_3')>, <KerasTensor: shape=(None, 6) dtype=float32 (created by layer 'string_lookup_12')>, <KerasTensor: shape=(None, 4) dtype=float32 (created by layer 'string_lookup_13')>, <KerasTensor: shape=(None, 4) dtype=float32 (created by layer 'string_lookup_14')>, <KerasTensor: shape=(None, 33) dtype=float32 (created by layer 'string_lookup_15')>]


In [49]:
preprocesssed_result = tf.concat(preprocessed, axis=-1)
preprocesssed_result

<KerasTensor: shape=(None, 48) dtype=float32 (created by layer 'tf.concat_2')>

In [50]:
preprocessor = tf.keras.Model(inputs, preprocesssed_result)

In [51]:
body = tf.keras.Sequential([
  tf.keras.layers.Dense(10, activation='relu'),
  tf.keras.layers.Dense(10, activation='relu'),
  tf.keras.layers.Dense(1)
])

In [52]:
x = preprocessor(inputs)
x

<KerasTensor: shape=(None, 48) dtype=float32 (created by layer 'model_4')>

In [53]:
result = body(x)
result

<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'sequential_2')>

In [57]:
model = tf.keras.Model(inputs, result)

model.compile(optimizer='adam',
                loss=tfr.keras.losses.ListMLELoss(),
                metrics=['accuracy'])

In [59]:
class RankingModel(tfrs.Model):

  def __init__(self, loss):
    super().__init__()

    # Compute predictions.
    self.ranking_model = tf.keras.Sequential([
      # Learn multiple dense layers.
      tf.keras.layers.Dense(256, activation="relu"),
      tf.keras.layers.Dense(64, activation="relu"),
      # Make rating predictions in the final layer.
      tf.keras.layers.Dense(1)
    ])

    self.task = tfrs.tasks.Ranking(
      loss=loss,
      metrics=[
        tfr.keras.metrics.NDCGMetric(name="ndcg_metric"),
        tf.keras.metrics.RootMeanSquaredError()
      ]
    )

  def call(self, features):
    # We first convert the id features into embeddings.
    model_embeddings = self.model_embeddings(features["model"])

    tuning_embeddings = self.tuning_embeddings(features["tuning"])

    # We want to concatenate model embeddings with tuning emebeddings to pass
    # them into the ranking model. To do so, we need to reshape the model
    # embeddings to match the shape of tuning embeddings.
    list_length = features["tuning"].shape[1]
    model_embedding_repeated = tf.repeat(
        tf.expand_dims(model_embeddings, 1), [list_length], axis=1)

    # Once reshaped, we concatenate and pass into the dense layers to generate
    # predictions.q
    concatenated_embeddings = tf.concat(
        [model_embedding_repeated, tuning_embeddings], 2)

    return self.ranking_model(concatenated_embeddings)

  def compute_loss(self, features, training=False):
    labels = features.pop("ranking")

    scores = self(features)

    return self.task(
        labels=labels,
        predictions=tf.squeeze(scores, axis=-1),
    )

In [60]:
listwise_model = RankingModel(inputs, result, tfr.keras.losses.ListMLELoss())
listwise_model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))
listwise_model.fit(dict(df), epochs=10, verbose=True)

TypeError: RankingModel.__init__() takes 2 positional arguments but 4 were given

In [58]:
history = model.fit(dict(df), target, epochs=2, batch_size=2)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type list).

In [129]:
class MyModel(tf.keras.Model):
  def __init__(self):
    # Create all the internal layers in init.
    super().__init__(self)

    self.normalizer = tf.keras.layers.Normalization(axis=-1)

    self.seq = tf.keras.Sequential([
      self.normalizer,
      tf.keras.layers.Dense(10, activation='relu'),
      tf.keras.layers.Dense(10, activation='relu'),
      tf.keras.layers.Dense(1)
    ])

  def adapt(self, inputs):
    # Stack the inputs and `adapt` the normalization layer.
    inputs = stack_dict(inputs)
    self.normalizer.adapt(inputs)

  def call(self, inputs):
    # Stack the inputs
    inputs = stack_dict(inputs)
    # Run them through all the layers.
    result = self.seq(inputs)

    return result

model = MyModel()

model.adapt(inputs)

model.compile(optimizer='adam',
              loss=tfr.keras.losses.ListMLELoss(),
              metrics=['accuracy'],
              run_eagerly=True)

TypeError: int() argument must be a string, a bytes-like object or a real number, not 'NoneType'

In [113]:
model.fit(dict(numeric_features), target, epochs=5, batch_size=2)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).

In [104]:
ratings = tfds.load("movielens/100k-ratings", split="train")
print(type(ratings))
movies = tfds.load("movielens/100k-movies", split="train")

ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
    "user_rating": x["user_rating"],
})
print(type(ratings))
movies = movies.map(lambda x: x["movie_title"])

unique_movie_titles = np.unique(np.concatenate(list(movies.batch(1000))))
unique_user_ids = np.unique(np.concatenate(list(ratings.batch(1_000).map(
    lambda x: x["user_id"]))))

<class 'tensorflow.python.data.ops.prefetch_op._PrefetchDataset'>
<class 'tensorflow.python.data.ops.map_op._MapDataset'>


In [90]:
tf.random.set_seed(42)

# Split between train and tests sets, as before.
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

# We sample 50 lists for each user for the training data. For each list we
# sample 5 movies from the movies the user rated.
train = tfrs.examples.movielens.sample_listwise(
    train,
    num_list_per_user=50,
    num_examples_per_list=5,
    seed=42
)
test = tfrs.examples.movielens.sample_listwise(
    test,
    num_list_per_user=1,
    num_examples_per_list=5,
    seed=42
)

In [91]:
import pprint
for example in train.take(1):
  pprint.pprint(example)

{'movie_title': <tf.Tensor: shape=(5,), dtype=string, numpy=
array([b'Postman, The (1997)', b'Liar Liar (1997)', b'Contact (1997)',
       b'Welcome To Sarajevo (1997)',
       b'I Know What You Did Last Summer (1997)'], dtype=object)>,
 'user_id': <tf.Tensor: shape=(), dtype=string, numpy=b'681'>,
 'user_rating': <tf.Tensor: shape=(5,), dtype=float32, numpy=array([4., 5., 1., 4., 1.], dtype=float32)>}


In [99]:
class RankingModel(tfrs.Model):

  def __init__(self, loss):
    super().__init__()
    embedding_dimension = 32

    # Compute embeddings for users.
    self.user_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_user_ids),
      tf.keras.layers.Embedding(len(unique_user_ids) + 2, embedding_dimension)
    ])

    # Compute embeddings for movies.
    self.movie_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_movie_titles),
      tf.keras.layers.Embedding(len(unique_movie_titles) + 2, embedding_dimension)
    ])

    # Compute predictions.
    self.score_model = tf.keras.Sequential([
      # Learn multiple dense layers.
      tf.keras.layers.Dense(256, activation="relu"),
      tf.keras.layers.Dense(64, activation="relu"),
      # Make rating predictions in the final layer.
      tf.keras.layers.Dense(1)
    ])

    self.task = tfrs.tasks.Ranking(
      loss=loss,
      metrics=[
        tfr.keras.metrics.NDCGMetric(name="ndcg_metric"),
        tf.keras.metrics.RootMeanSquaredError()
      ]
    )

  def call(self, features):
    # We first convert the id features into embeddings.
    # User embeddings are a [batch_size, embedding_dim] tensor.
    user_embeddings = self.user_embeddings(features["user_id"])

    # Movie embeddings are a [batch_size, num_movies_in_list, embedding_dim]
    # tensor.
    movie_embeddings = self.movie_embeddings(features["movie_title"])

    # We want to concatenate user embeddings with movie emebeddings to pass
    # them into the ranking model. To do so, we need to reshape the user
    # embeddings to match the shape of movie embeddings.
    list_length = features["movie_title"].shape[1]
    user_embedding_repeated = tf.repeat(
        tf.expand_dims(user_embeddings, 1), [list_length], axis=1)

    # Once reshaped, we concatenate and pass into the dense layers to generate
    # predictions.q
    concatenated_embeddings = tf.concat(
        [user_embedding_repeated, movie_embeddings], 2)

    return self.score_model(concatenated_embeddings)

  def compute_loss(self, features, training=False):
    labels = features.pop("user_rating")

    scores = self(features)

    return self.task(
        labels=labels,
        predictions=tf.squeeze(scores, axis=-1),
    )

In [100]:
epochs = 30

cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

In [103]:
listwise_model = RankingModel(tfr.keras.losses.ListMLELoss())
listwise_model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))
listwise_model.fit(cached_train, epochs=epochs, verbose=True)

Epoch 1/30
6/6 [==============================] - 3s 133ms/step - ndcg_metric: 0.8545 - root_mean_squared_error: 3.7307 - loss: 4.7866 - regularization_loss: 0.0000e+00 - total_loss: 4.7866
Epoch 2/30
6/6 [==============================] - 1s 129ms/step - ndcg_metric: 0.8644 - root_mean_squared_error: 3.7196 - loss: 4.7845 - regularization_loss: 0.0000e+00 - total_loss: 4.7845
Epoch 3/30
6/6 [==============================] - 1s 145ms/step - ndcg_metric: 0.8699 - root_mean_squared_error: 3.7112 - loss: 4.7821 - regularization_loss: 0.0000e+00 - total_loss: 4.7821
Epoch 4/30
6/6 [==============================] - 1s 122ms/step - ndcg_metric: 0.8737 - root_mean_squared_error: 3.7067 - loss: 4.7791 - regularization_loss: 0.0000e+00 - total_loss: 4.7791
Epoch 5/30
6/6 [==============================] - 1s 117ms/step - ndcg_metric: 0.8770 - root_mean_squared_error: 3.6981 - loss: 4.7748 - regularization_loss: 0.0000e+00 - total_loss: 4.7748
Epoch 6/30
6/6 [==============================] - 

In [67]:

import array
import collections

from typing import Dict, List, Optional, Text, Tuple

def _create_feature_dict() -> Dict[Text, List[tf.Tensor]]:
  """Helper function for creating an empty feature dict for defaultdict."""
  return {"encoder": [], "rank": []}

def _sample_list(
    feature_lists: Dict[Text, List[tf.Tensor]],
    num_examples_per_list: int,
    random_state: Optional[np.random.RandomState] = None,
) -> Tuple[tf.Tensor, tf.Tensor]:
  """Function for sampling a list example from given feature lists."""
  if random_state is None:
    random_state = np.random.RandomState()

  sampled_indices = random_state.choice(
      range(len(feature_lists["encoder"])),
      size=num_examples_per_list,
      replace=False,
  )
  sampled_movie_titles = [
      feature_lists["encoder"][idx] for idx in sampled_indices
  ]
  sampled_ratings = [
      feature_lists["rank"][idx]
      for idx in sampled_indices
  ]

  return (
      tf.stack(sampled_movie_titles, 0),
      tf.stack(sampled_ratings, 0),
  )

def sample_listwise(
    rating_dataset: tf.data.Dataset,
    num_list_per_user: int = 10,
    num_examples_per_list: int = 10,
    seed: Optional[int] = None,
) -> tf.data.Dataset:
  """Function for converting the MovieLens 100K dataset to a listwise dataset.

  Args:
      rating_dataset:
        The MovieLens ratings dataset loaded from TFDS with features
        "movie_title", "user_id", and "user_rating".
      num_list_per_user:
        An integer representing the number of lists that should be sampled for
        each user in the training dataset.
      num_examples_per_list:
        An integer representing the number of movies to be sampled for each list
        from the list of movies rated by the user.
      seed:
        An integer for creating `np.random.RandomState`.

  Returns:
      A tf.data.Dataset containing list examples.

      Each example contains three keys: "user_id", "movie_title", and
      "user_rating". "user_id" maps to a string tensor that represents the user
      id for the example. "movie_title" maps to a tensor of shape
      [sum(num_example_per_list)] with dtype tf.string. It represents the list
      of candidate movie ids. "user_rating" maps to a tensor of shape
      [sum(num_example_per_list)] with dtype tf.float32. It represents the
      rating of each movie in the candidate list.
  """
  random_state = np.random.RandomState(seed)

  example_lists_by_user = collections.defaultdict(_create_feature_dict)

  movie_title_vocab = set()
  for example in rating_dataset:
    user_id = example["dataset"].numpy()
    example_lists_by_user[user_id]["encoder"].append(
        example["encoder"])
    example_lists_by_user[user_id]["rank"].append(
        example["rank"])
    movie_title_vocab.add(example["encoder"].numpy())

  tensor_slices = {"dataset": [], "encoder": [], "rank": []}

  for user_id, feature_lists in example_lists_by_user.items():
    for _ in range(num_list_per_user):

      # Drop the user if they don't have enough ratings.
      if len(feature_lists["encoder"]) < num_examples_per_list:
        continue

      sampled_movie_titles, sampled_ratings = _sample_list(
          feature_lists,
          num_examples_per_list,
          random_state=random_state,
      )
      tensor_slices["dataset"].append(user_id)
      tensor_slices["encoder"].append(sampled_movie_titles)
      tensor_slices["rank"].append(sampled_ratings)

  return tf.data.Dataset.from_tensor_slices(tensor_slices)

test = sample_listwise(x, 1, 5, 42)
print(test)

TypeError: Cannot iterate over a Tensor with unknown first dimension.